
# Entrenamiento y ejecución de un pipeline de clasificación textual

La clasificación de textos consiste en, dado un texto, asignarle una entre varias categorías. Algunos ejemplos de esta tarea son:

- dado un tweet, categorizar su connotación como positiva, negativa o neutra.
- dado un post de Facebook, clasificarlo como portador de un lenguaje ofensivo o no.  

En la actividad exploraremos cómo crear un pipeline para la clasificación de textos, entrenarlo y utilizarlo para clasificar nuevos datos.

**Instrucciones:**

- siga las indicaciones y comentarios en cada apartado.

**Después de esta actividad nos habremos familiarizado con:**
- algunos tipos de carácterísticas ampliamente utilizadas en la clasificación de textos. 
- cómo contruir un pipeline para la clasificación de textos utilizando sklearn.
- utilizar este pipeline para clasificar nuevos textos.

**Requerimientos**
- python 3.6 - 3.8
- pandas
- plotly


## Instalación de librerías e importación de dependencias.

Para comenzar, es preciso instalar e incluir las librerías necesarias. En este caso, el entorno de Colab incluye las necesarias.

Ejecute la siguiente casilla prestando atención a las explicaciónes dadas en los comentarios.

In [ ]:
#  para construir gráficas y realizar análisis exploratorio de los datos
import plotly.graph_objects as go

# para cargar datos y realizar pre-procesamiento básico
import pandas as pd
from collections import Counter

# para pre-procesamiento del texto y extraer carácterísticas
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.stem.snowball import EnglishStemmer

# algorítmos de clasificación
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# para construir pipelines
from sklearn.pipeline import Pipeline

# para evaluar los modelos 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score  

# para guardar el modelo
import pickle

print('Done!')

## Definición de funciones y variables necesarias para el preprocesamiento de datos

Antes de definir el pipeline definiremos algunas variables útiles como el listado de stop words y funciones para cargar los datos, entrenar el modelo etc.

In [ ]:

#listado de stopwords. Este listado también se puede leer desde un fichero utilizando la función read_corpus
stop_words=['i','me','my','myself','we','our','ours','ourselves','you','your','yours','yourself','yourselves',
            'he','him','his','himself','she','her','hers','herself','it','its','itself','they','them','their',
            'theirs','themselves','what','which','who','whom','this','that','these','those','am','is','are',
            'was','were','be','been','being','have','has','had','having','do','does','did','doing','a','an',
            'the','and','but','if','or','because','as','until','while','of','at','by','for','with','about',
            'against','between','into','through','during','before','after','above','below','to','from','up',
            'down','in','out','on','off','over','under','again','further','then','once','here','there','when',
            'where','why','how','all','any','both','each','few','more','most','other','some','such','no','nor',
            'not','only','own','same','so','than','too','very','s','t','can','will','just','don','should','now', 'ever']


# obtiene un dataframe de pandas
def read_corpus(file, sep):
    return pd.read_csv(file, sep)


# muestra gráficamente la distribución de clases del conjunto de datos utilizado
def plot(corpus):
    dist = corpus.groupby(["Sentiment"]).size()
    dist = dist / dist.sum()* 100
    fig, ax = plt.subplots(figsize=(12, 8))
    sns.barplot(dist.keys(), dist.values);
    plt.show()


# función auxiliar utilizada por CountVectorizer para procesar las frases
def english_stemmer(sentence):
    stemmer = EnglishStemmer()
    analyzer = CountVectorizer(binary=False, analyzer='word', stop_words=stop_words,
                               ngram_range=(1, 1)).build_analyzer()
    return (stemmer.stem(word) for word in analyzer(sentence))


# entrena el pipeline
def fit_model(data, target, pipeline):
    pipeline.fit(data, target)


# utiliza el pipeline para predecir datos
def predict(data_test, pipeline):
    return pipeline.predict(data_test)


# evalua el pipeline entrenado de acuerdo a una de las métricas apropiadas para un problema de clasificación. 
def evaluate_model(predicted, target_test):
    print(classification_report(target_test, predicted))
    print("The accuracy score is {:.2%}".format(accuracy_score(target_test, predicted)))


# guarda un pipeline entrenado
def saveModel(model, modelName = "pickle_model.pkl"):
   pkl_filename = modelName
   with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)   


# carga un pipeline entrenado y guardado previamente
def loadModel(rutaModelo = "pickle_model.pkl"):
  # Load from file
  with open(rutaModelo, 'rb') as file:
    pickle_model = pickle.load(file)
    return pickle_model 

## Creación de un pipeline para la clasificación de textos.

Para construir el pipeline, utilizaremos la clase Pipeline de sklean. Esta permite encadenar los diferentes pasos, por ejemplo, algoritmos de extracción de características y un clasificador. Por ejemplo, para obtener un pipeline que comprende CountVectorizer, seguido de TfidfTransformer y un Support Vector Machine como clasificador, se utilizaría esta sentencia:

~~~ 
Pipeline([
        ('dataVect', CountVectorizer(analyzer=english_stemmer)),
        ('tfidf', TfidfTransformer(smooth_idf=True, use_idf=True)),
        (classifier, SVC(probability=True) )
     ])
~~~

Para tener mayor flexibilidad si se desean probar varios clasificadores, podría construirse el pipeline sin clasificador, incluyendo este con posterioridad. Este será el enfoque que seguiremos en la actividad.

Ejecute la siguiente casilla para definir una función que construye un pipeline con las características antes mencionadas.


In [ ]:
def preprocessPipeLine():
    return Pipeline([
        ('dataVect', CountVectorizer(analyzer=english_stemmer)),
        ('tfidf', TfidfTransformer(smooth_idf=True, use_idf=True)),
     ])


## Carga de datos y análisis exploratorio.

Antes de entrenar el pipeline, es necesario cargar los datos. Existen diferentes opciones, entre estas:

- montar nuestra partición de Google Drive y leer un fichero desde esta.

- leer los datos desde un fichero en una carpeta local.

- leer los datos directamente de un URL.

Ejecute la siguiente casilla prestando atención a las instrucciones adicionales en los comentarios.


In [ ]:
# descomente las siguientes 2 líneas para leer datos desde Google Drive,sumiendo que se trata de un fichero llamado review.csv localizado dentro de una carpeta llamada 'Datos' en su Google Drive.
from google.colab import drive
drive.mount('/content/drive')
corpus = pd.read_csv('/content/drive/MyDrive/Datos/review.csv', sep=',')

# descomente la siguiente línea para leer los datos desde un archivo local, por ejemplo, asumiendo que se encuentra dentro de un directorio llamado sample_data
#corpus = pd.read_csv('/sample_data/review.csv', sep=',')

# descomente la siguiente línea para leer datos desde un URL
#corpus = pd.read_csv('/content/sample_data/review.csv')


# graficar distribución de clases en los datos
colors = ['darkgreen', 'red']
categories = sorted(corpus['Sentiment'].unique())
hist= Counter(corpus['Sentiment'])

fig = go.Figure(layout=go.Layout(height=400, width=600))
fig.add_trace(go.Bar(x=categories, y=[hist[cat] for cat in sorted(hist.keys())], marker_color=colors))
fig.show()

print('Done!')

## Definición de una función para el entrenamiento del modelo

Ejecute la siguiente casilla para definir la función *main*  que integra todas las funciones definidas para constuir el pipeline, entrenarlo, evaluarlo y guardarlo para su posterior uso.


In [ ]:
def main():  
    
    # crear el pipeline (solo incluyendo los pasos de pre-procesamiento)
    pipeline=preprocessPipeLine()
    
    # crear el clasificador y añadirlo al pipeline. Puede probar diferentes clasificadores
    # classifier = MultinomialNB()
    # classifier = DecisionTreeClassifier()
    classifier = SVC(probability=True)

    pipeline.steps.append(('classifier', classifier))

    # preparar conjuntos de entrenamiento (60%) y prueba (40%)
    data_train, data_test, target_train, target_test = train_test_split(corpus['Phrase'].values, corpus['Sentiment'].values, test_size=0.4, random_state=43)

    # entrenar el modelo
    fit_model(data_train, target_train, pipeline)

    # predecir y evaluar el modelo en el conjunto de entrenamiento
    print('=== Evaluación conjunto de entrenamiento ====')
    predicted = predict(data_train, pipeline)
    evaluate_model(predicted, target_train)

    # predecir y evaluar el modelo en el conjunto de prueba
    print('\n=== Evaluación conjunto de entrenamiento ====')
    predicted = predict(data_test, pipeline)
    evaluate_model(predicted, target_test)

    # guardar el modelo
    saveModel(pipeline)
  

Finalmente, llamamos a la función *main* para entrenar el pipeline, evaluarlo y guardarlo.

In [ ]:
main()

# Predicción de nuevos datos

Una vez que disponemos de un pipeline entrenado y guardado, podemos cargarlo para predecir nuevos datos.

Ejecute la siguiente casilla prestando atención a las explicaciones dadas en los comentarios.

In [ ]:
# datos a predecir. Notar que estos podrian obtenerse desde un fichero de modo similar a como se cargaron los datos de entrenamiento.
# Notar que para predecir nuevos datos no es necesario conocer su clasificación, !para esto utilizaremos el pipeline entrenado!
Xtest = ["The movie was awesome!","Martin Campbell has produced better novels.","I liked the views but the rest of the movie was awful."]

# cargar pipeline entrenado
pickle_model = loadModel()

# predecir los nuevos datos
print(pickle_model.predict(Xtest))

# predecir los nuevos datos, mostrando la probabilidad de la clasificación
print(pickle_model.predict_proba(Xtest))
